### 베이지안 확률

In [16]:
import pandas as pd
import string
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
import nltk

# 정상메일과 스펨메일을 구분하기 위해 다음과 같은 데이터셋을 사용

data = [["1", "spam", "your free lottery", 1],
       ["2", "spam", "free lottery free you", 1],
       ["3", "ham", "your free apple", 0],
       ["4", "ham", "free to contact me", 0],
       ["5", "ham", "I won award", 0],
       ["6", "spam", "my lottery ticket", 1]]

df = pd.DataFrame(data, columns = ["num", "label", "text", "label_num"])

df

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ChaeEun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,num,label,text,label_num
0,1,spam,your free lottery,1
1,2,spam,free lottery free you,1
2,3,ham,your free apple,0
3,4,ham,free to contact me,0
4,5,ham,I won award,0
5,6,spam,my lottery ticket,1


In [25]:
def process_text(text):
    
    # text에서 구두점을 삭제
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc) # join: 리스트에서 문자열로
    
    # text에서 무의미한 단어(접미사, 조사 등)는 삭제
    cleaned_words = [word for word in nopunc.split()
                    if word.lower() not in stopwords.words('english')]
    return cleaned_words

# process_text 함수를 적용하여 데이터 세트의 텍스트 데이터를 토큰화
df['text'].head().apply(process_text)

# text를 토큰 수의 행렬로 변환
from sklearn.feature_extraction.text import CountVectorizer

messages_bow = CountVectorizer(analyzer = process_text).fit_transform(df['text'])

# 데이터를 80%의 training과 20%의 testing 데이터셋으로 분리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(messages_bow, df["label_num"], test_size = 0.20, random_state = 0)

# 다항식 나이브베이즈 모델로 만들고 훈련
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(x_train, y_train)

# 데이터셋 분류에 대한 예측 및 실제 관측 값을 보여주기
print(classifier.predict(x_train)) # 예측 값 출력
print(y_train.values) # 실제 관측 값 출력

[1 0 1 0]
[1 0 1 0]


In [26]:
# 학습 데이터셋에서 모델의 정확도를 표현
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

pred = classifier.predict(x_train) # 예측 값 출력

# 사이킷런 패키지의 metrics 패키지는 정밀도, 재현율, F1 점수를 구함
print(classification_report(y_train, pred))

# 혼동행렬로 표현
print("Confusion Matrix: \n", confusion_matrix(y_train, pred))
print()
print("Accuracy: ", accuracy_score(y_train, pred)) # 정확도 점수로 표현

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

Confusion Matrix: 
 [[2 0]
 [0 2]]

Accuracy:  1.0


In [27]:
# 테스트 데이터셋(x_test & y_test)에서 모델의 정확도를 테스트
print("Predicted value: ", classifier.predict(x_test))

Predicted value:  [1 1]


In [28]:
# 실제 관측 값 출력
print("Actual value: ", y_test.values)

Actual value:  [1 0]


In [30]:
# 테스트 데이터셋에서 모델을 평가
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score

pred = classifier.predict(x_test)
print(classification_report(y_test, pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, pred))
print()
print("Accuracy: ", accuracy_score(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Confusion Matrix: 
 [[0 1]
 [0 1]]

Accuracy:  0.5


/Users/ChaeEun/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
